In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

def bias(x, y): return np.mean(x - y)

from spicy_snow.retrieval import retrieval_from_parameters

In [11]:
lievens_params = [2, 0.5, 0.44]
wus_params = [2.5, 0.2, 0.55]

in_dir = Path('~/spicy-snow/SnowEx-Data/').expanduser().resolve()
data_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()

# Create parameter space
A = np.round(np.arange(1, 3.1, 0.1), 2)
B = np.round(np.arange(0, 2.01, 0.1), 2)
C = np.round(np.arange(0, 1.001, 0.01), 2)

def get_stats(x, y):
    if type(x) == xr.DataArray: x = x.values.ravel()
    if type(y) == xr.DataArray: y = y.values.ravel()
    idx = (~np.isnan(x)) & (~np.isnan(y))
    x, y = x[idx], y[idx]
    r, p = pearsonr(x, y)
    b = bias(x, y)
    mae = mean_absolute_error(x, y)
    rmse = mean_squared_error(x, y, squared = False)
    return r, b, mae, rmse

In [7]:
res = pd.DataFrame()

for fp in in_dir.glob('*.nc'):
    ds = xr.open_dataset(fp)

    if fp.stem == 'Frasier_2020-02-11':
        r, b, mae, rmse = get_stats(ds['lidar-sd'], ds['snow_depth'].sel(time = '2020-02-16'))
        im_date = pd.to_datetime('2020-02-16')
    else:
        r, b, mae, rmse = get_stats(ds['lidar-sd'], ds['snow_depth'].sel(time = ds.attrs['lidar-flight-time'], method = 'nearest'))
        im_date = pd.to_datetime(ds.sel(time = ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])
    d_days = im_date - pd.to_datetime(ds.attrs['lidar-flight-time'])

    site_name = fp.stem.replace('_', ' ')
    for name, var in zip(['RMSE', 'bias', 'MAE', 'Pearson R'], [rmse, b, mae, r]):
        res.loc[site_name, name] = var

    new_ds = retrieval_from_parameters(xr.open_dataset(data_dir.joinpath(fp.name)), A = lievens_params[0], B = lievens_params[1], C = lievens_params[2])

    o_r, o_b, o_mae, o_rmse = get_stats(new_ds['lidar-sd'], new_ds['snow_depth'].sel(time = im_date, method = 'nearest'))

    res.loc[site_name, '△ rmse'] = rmse - o_rmse

In [9]:
o_rmse

1.133012564454194

In [10]:
rmse

1.0980119588745152

In [8]:
res

,RMSE,bias,MAE,Pearson R,△ rmse
Mores 2020-02-09,1.118619,0.169999,0.933682,0.028640,0.052566
Frasier 2021-03-19,0.723593,0.217063,0.580364,0.112713,0.084840
Dry Creek 2020-02-19,0.851786,-0.099454,0.644916,0.155150,0.188747
Banner 2021-03-15,1.370068,-0.541134,1.003033,0.309225,0.510551
Little Cottonwood 2021-03-18,1.479392,-0.319309,1.154803,0.314687,0.290218
Mores 2021-03-15,1.133725,-0.060238,0.903184,0.355782,0.209520
Banner 2020-02-18,0.998748,0.324339,0.832388,0.289871,0.033294
Frasier 2020-02-11,0.950128,0.650994,0.831223,0.246458,0.006630
Cameron 2021-03-19,1.098012,0.642722,0.936709,0.056289,-0.035001
